## 1. 类

In [1]:
import datetime

class Person(object):
    def __init__(self, name):
        """创建一个人"""
        self.name = name
        try:
            lastBlank = name.rindex(' ')
            self.lastName = name[lastBlank+1:]
        except:
            self.lastName = name
            self.birthday = None
            
    def getName(self):
        """返回self的全名"""
        return self.name
    
    def getLastName(self):
        """返回self的姓"""
        return self.lastName
    
    def setBirthday(self, birthdate):
        """假设birthday是datetime.date类型
        将self的生日设置为birthday"""
        self.birthday = birthdate
        
    def getAge(self):
        """返回self的当前年龄，用日表示"""
        if self.birthday == None:
            raise ValueError
        return (datetime.date.today() - self.birthday).days

    def __lt__(self, other):
        """如果self按字母顺序位于other之前，则返回True，否则返回False。
        首先按照姓进行比较，如果姓相同，就按照全名比较"""

        if self.lastName == other.lastName:
            #  self.name < other.name 是 self.name.__lt__(other.name)
            return self.name < other.name
        return self.lastName < other.lastName

    # 执行print命令时，会自动调用与待输出对象相关联的__str__方法
    # print(类的对象)，打印出__str__中的内容
    def __str__(self):
        """返回self的全名"""
        return self.name

In [2]:
me = Person('Michael Guttag')
him = Person('Barack Hussein Obama')
her = Person('Madonna')
print(him.getLastName())

him.setBirthday(datetime.date(1961, 8, 4))
her.setBirthday(datetime.date(1958, 8, 16))
print(him.getName(), 'is', him.getAge(), 'days old')

Obama
Barack Hussein Obama is 21033 days old


## 2. 继承

In [11]:
class MITPerson(Person):
    nextIdNum = 0 #identification number
    def __init__(self, name):
        Person.__init__(self, name)
        self.idNum = MITPerson.nextIdNum
        MITPerson.nextIdNum += 1
        
    def getIdNum(self):
        return self.idNum
    
    def isStudent(self):
        return isinstance(self, Student)
    
    def __lt__(self, other):
        return self.idNum < other.idNum

In [4]:
p1 = MITPerson('Barbara Beaver')
print(str(p1) + '\'s id number is ' + str(p1.getIdNum()))

Barbara Beaver's id number is 0


In [17]:
p1 = MITPerson('Mark Guttag')
p2 = MITPerson('Billy Bob Beaver')
p3 = MITPerson('Billy Bob Beaver')
p4 = Person('Billy Bob Beaver')

In [6]:
print('p1 < p2 =', p1 < p2)
print('p3 < p2 =', p3 < p2)

# 调用哪种__lt__方法是由表达式的第一个参数决定的，p4 < p1是p4.__lt__(p1)的简写，
# 所以解释器使用与p4的类型Person关联的__lt__方法，按照名字排序

print('p4 < p1 =', p4 < p1)

p1 < p2 = True
p3 < p2 = False
p4 < p1 = True


## 3. 多重继承

In [14]:
class Student(MITPerson):
    pass


class UG(Student):
    def __init__(self, name, classYear):
        MITPerson.__init__(self, name)
        # super().__init__(name)
        self.year = classYear
        
    def getClass(self):
        return self.year
    

class Grad(Student):
    pass

In [15]:
p5 = Grad('Buzz Aldrin')
p6 = UG('Billy Beaver', 1984)
print(p5, 'is a graduate student is', type(p5) == Grad)
print(p5, 'is an undergraduate student is', type(p5) == UG)

Buzz Aldrin is a graduate student is True
Buzz Aldrin is an undergraduate student is False


In [18]:
print(p5, 'is a student is', p5.isStudent())
print(p6, 'is a student is', p6.isStudent())

# p3 = MITPerson('Billy Bob Beaver')
print(p3, 'is a student is', p3.isStudent())

Buzz Aldrin is a student is True
Billy Beaver is a student is True
Billy Bob Beaver is a student is False


In [19]:
class TransferStudent(Student):
    def __init__(self, name, fromSchool):
        MITPerson.__init__(self, name)
        self.fromSchool = fromSchool
        
    def getOldSchool(self):
        return self.fromSchool

## 4. 封装与信息隐藏

In [21]:
class Grades(object):
    def __init__(self):
        self.students = []
        self.grades = {}
        self.isSorted = True
        
    def addStudent(self, student):
        """假设student为Student类型
            将student添加到成绩册"""
        if student in self.students:
            raise ValueError("Duplicate student")
        self.students.append(student)
        self.grades[student.getIdNum()] = []
        self.isSorted = False
        
    def addGrade(self, student, grade):
        """假设grade为浮点数
           将grade添加到student的成绩列表"""
        try:
            self.grades[student.getIdNum()].append(grade)
        except:
            raise ValueError('Student not in mapping')
            
    def getGrades(self, student):
        """返回student的成绩列表"""
        try: #return copy of list of student's grades
            return self.grades[student.getIdNum()][:]
        except:
            raise ValueError('Student not in mapping')
            
    def getStudents(self):
        """返回成绩册中排好序的成绩列表"""
        if not self.isSorted:
            self.students.sort()
            self.isSorted = True
        return self.students[:]#返回一个学生列表的副本

In [24]:
def gradeReport(course):
    """假设course是Grades类型"""
    report = ''
    for s in course.getStudents():
        tot = 0.0
        numGrades = 0
        for g in course.getGrades(s):
            tot += g
            numGrades += 1
        try:
            average = tot/numGrades
            report = report + '\n'\
            + str(s) + '\'s mean grade is ' + str(average)
        except ZeroDivisionError:
            report = report + '\n'\
            + str(s) + ' has no grades'
    return report


ug1 = UG('Jane Doe', 2014)
ug2 = UG('John Doe', 2015)
ug3 = UG('David Henry', 2003)

g1 = Grad('Billy Buckner')
g2 = Grad('Bucky F. Dent')

sixHundred = Grades()
sixHundred.addStudent(ug1)
sixHundred.addStudent(ug2)
sixHundred.addStudent(g1)
sixHundred.addStudent(g2)
for s in sixHundred.getStudents():
    sixHundred.addGrade(s, 75)
sixHundred.addGrade(g1, 25)
sixHundred.addGrade(g2, 100)
sixHundred.addStudent(ug3)
print(gradeReport(sixHundred))


Jane Doe's mean grade is 75.0
John Doe's mean grade is 75.0
David Henry has no grades
Billy Buckner's mean grade is 50.0
Bucky F. Dent's mean grade is 87.5


## 5. 封装与信息隐藏

In [26]:
class infoHiding(object):
    def __init__(self):
        self.visible = 'Look at me'
        self.__alsoVisible__ = 'Look at me too'
        # 类的私有变量
        self.__invisible = 'Don\'t look at me directly'
        
    def printVisible(self):
        print(self.visible)
        
    # 使用方法可以访问类的私有变量
    def printInvisible(self):
        print(self.__invisible)
        
    # 类的私有方法
    def __printInvisible(self):
        print(self.__invisible)
        
    def __printInvisible__(self):
        print(self.__invisible)

In [27]:
test = infoHiding()
print(test.visible)
print(test.__alsoVisible__)
print(test.__invisible)

Look at me
Look at me too


AttributeError: 'infoHiding' object has no attribute '__invisible'

In [28]:
test = infoHiding()
test.printInvisible()
test.__printInvisible__()
test.__printInvisible()

Don't look at me directly
Don't look at me directly


AttributeError: 'infoHiding' object has no attribute '__printInvisible'

In [29]:
class subClass(infoHiding):
    def __init__(self):
        print('from subclass', self.__invisible)
        
testSub = subClass()

AttributeError: 'subClass' object has no attribute '_subClass__invisible'

## 6. 生成器

In [31]:
def getStudents(self):
    """按字母顺序每次返回成绩册中的一个学生in alphabetical order"""
    if not self.isSorted:
        self.students.sort()
        self.isSorted = True
    for s in self.students:
        yield s

In [32]:
ook = Grades()
book.addStudent(Grad('Julie'))
book.addStudent(Grad('Charlie'))
for s in book.getStudents():
    print(s)

NameError: name 'book' is not defined